In [1]:
import social_interaction
import code_interaction
import buggy_commit
import pandas as pd
import numpy as np
import csv
import platform
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

In [2]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = 'Processed Data/'
else:
    _dir = 'Processed Data\\'
Files = [join(_dir, f) for f in listdir(_dir) if isfile(join(_dir, f))]
actual_file = 0
usable_files = 0
size_s = 0
size_m = 0
size_l = 0
result = []
for file in Files:
    _df = []
    actual_file += 1
    df = pd.read_pickle(file)
    buggy_commit_data_df = df.loc[0,'buggy_commit_data_df']
    commit_data_df = df.loc[0,'commit_data_df']
    commit_count = df.loc[0,'commit_count']
    sg_data_df = df.loc[0,'sg_data_df']
    cg_data_df = df.loc[0,'cg_data_df']
    if commit_data_df.shape[0] < 8:
        size_s += 1
    elif commit_data_df.shape[0] < 15:
        size_m += 1
    else:
        size_l += 1
    if commit_data_df.shape[0] < 8:
        continue
    for i in range(sg_data_df.shape[0]):
        buggy_commit_count = buggy_commit_data_df[buggy_commit_data_df['committer'] == sg_data_df.loc[i,'committer']]['count']
        if len(buggy_commit_count) == 0:
            continue
        commit_count = commit_data_df[commit_data_df['committer'] == sg_data_df.loc[i,'committer']]['count']
        if len(commit_count) == 0:
            continue
        node_degree = sg_data_df[sg_data_df['committer'] == sg_data_df.loc[i,'committer']]['count']
        #print([buggy_commit_count.values[0],commit_count.values[0],node_degree.values[0]])
        _df.append([buggy_commit_count.values[0]/commit_count.values[0],node_degree.values[0]])
    df = pd.DataFrame(_df, columns = ['per','degree'])
    if(df.shape[0] <= 0):
        continue
    usable_files += 1
    degree_d = np.array(df['degree'].values.tolist())
    #first = np.int32(np.percentile(degree_d,99.9))
    #second = np.int32(np.percentile(degree_d,99))
    #third = np.int32(np.percentile(degree_d,80))
    forth = np.int32(np.percentile(degree_d,95))
    first_l = []
    second_l = []
    third_l = []
    forth_l = []
    for i in range(df.shape[0]):
        if df.loc[i,'degree'] < forth:
            forth_l.append(df.loc[i,'per'])
        #elif df.loc[i,'degree'] < third:
        #    third_l.append(df.loc[i,'per'])
        #elif df.loc[i,'degree'] < second:
        #    second_l.append(df.loc[i,'per'])
        else:
            first_l.append(df.loc[i,'per'])
    #print(round(np.median(first_l),2),round(np.median(second_l),2),round(np.median(third_l),2),round(np.median(forth_l),2))
    result.append([commit_data_df.shape[0],round(np.median(first_l),2),round(np.median(forth_l),2)])
print(actual_file,usable_files)
print(size_s,size_m,size_l)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [14]:
result_df = pd.DataFrame(result, columns = ['size','experienced','inexperienced'])

In [15]:
result_df.to_excel('Result_data_team_s.xlsx')